Домашнее задание
1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

Ссылки
https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/
https://en.wikipedia.org/wiki/Receiver_operating_characteristic

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

# df = pd.read_csv('train_case2.csv', ';')
df = pd.read_csv('/content/sample_data/train_case2.csv', ';')
df.head(3)



,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [2]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [3]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [4]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)

# feature_processing = Pipeline([('feats', feats)])
# feature_processing.fit_transform(X_train)

Добавим классификатор и запустим кросс-валидацию для моделей логистической LogisticRegression, RandomForest и XGBoost

Обернем кроссвалидацию в функцию

In [5]:
def my_cv(model, X_train, y_train):
    #запустим кросс-валидацию
    cv_scores = cross_val_score(model, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print('CV score is {}+-{}'.format(cv_score, cv_score_std))

In [6]:
LR_cls = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])
my_cv(LR_cls, X_train, y_train)

CV score is 0.7867401104915408+-0.00852135511666111


In [7]:
RF_cls = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state=42,
                                         max_depth=10)),
])
my_cv(RF_cls, X_train, y_train)

CV score is 0.8012814315210153+-0.006725085121681367


In [8]:
XGB_cls = Pipeline([
    ('features',feats),
    ('classifier', XGBClassifier(random_state = 42,
                                 eta=0.3,
                                 max_depth=4)),
])
my_cv(XGB_cls, X_train, y_train)

CV score is 0.8030305340079853+-0.007090010275957017


In [9]:
# оформим в функцию расчет метрик
def metrics(y_test, preds):
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix]))
    return precision, recall, thresholds, ix, fscore

In [10]:
LR_cls.fit(X_train, y_train)
RF_cls.fit(X_train, y_train)
XGB_cls.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('age',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler(copy=True,
                                                                                 with_mean=True,
                                                                                 with_std=True))],
                                                          verbose=False)),
                                                ('height',
                                                 Pipeline(memory=None,
                                    

In [11]:
y_score_LR = LR_cls.predict_proba(X_test)[:, 1]
y_score_RF = RF_cls.predict_proba(X_test)[:, 1]
y_score_XGB = XGB_cls.predict_proba(X_test)[:, 1]

In [12]:
precision_LR, recall_LR, thresholds_LR, ix_LR, fscore_LR = metrics(y_test, y_score_LR)
precision_RF, recall_RF, thresholds_RF, ix_RF, fscore_RF = metrics(y_test, y_score_RF)
precision_XGB, recall_XGB, thresholds_XGB, ix_XGB, fscore_XGB = metrics(y_test, y_score_XGB)

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838
Best Threshold=0.400248, F-Score=0.740, Precision=0.697, Recall=0.790
Best Threshold=0.371986, F-Score=0.740, Precision=0.682, Recall=0.809


In [13]:
roc_auc_score_LR = roc_auc_score(y_test, y_score_LR)
roc_auc_score_LR

0.7840347790421852

In [14]:
roc_auc_score_RF = roc_auc_score(y_test, y_score_RF)
roc_auc_score_RF

0.8015363399584103

In [15]:
roc_auc_score_XGB = roc_auc_score(y_test, y_score_XGB)
roc_auc_score_XGB

0.803003366092981

In [16]:
report = pd.DataFrame(
    {
        'Precision': [precision_LR[ix_LR], precision_RF[ix_RF], precision_XGB[ix_XGB]],
        'Recall': [recall_LR[ix_LR], recall_RF[ix_RF], recall_XGB[ix_XGB]],
        'F1-score': [fscore_LR[ix_LR], fscore_RF[ix_RF], fscore_XGB[ix_XGB]],
        'ROC AUC score': [roc_auc_score_LR, roc_auc_score_RF, roc_auc_score_XGB]
    },
    index=['LogisticRegression', 'RandomForest', 'XGBoost']
)

report

,Precision,Recall,F1-score,ROC AUC score
LogisticRegression,0.647431,0.837558,0.730323,0.784035
RandomForest,0.696545,0.789631,0.740173,0.801536
XGBoost,0.681708,0.809332,0.740058,0.803003


# Выводы

На самом деле все алгоритмы отработали достаточно хорошо. На основе F1 метрики линейнай регрессия уступает всего 1%, а roc_auc 1,5-2% При этом скорость логистической регрессии в разы выше, чем у более сложных алгоритмов Random Forest и тем более XGBoost.

Однако в точности выигрывает Random Forest с отрывом в 5% от регрессии и 1,5% от XGB. При этом полнота у него меньше всех, а в случае с логистической регрессией почти 5%.

Здесь надо уточнять, какие метрики наиболее важны. Но если базироваться на выявлении заболевания, то в этом случае нам важен высокий показатель recall, для большего охвата пациентов. Лучше провести для здоровых повторный анализ, чем пропустить заболевание. В таком раскладе обычная логистическая регрессия будет полезнее, как по скорости, так и по полноте.